In [1]:
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig

W1030 17:43:37.829000 9084 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1")
model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OSError: The paging file is too small for this operation to complete. (os error 1455)

In [3]:
dataset = load_dataset("json", data_files="hindi_wiki_llm_pairs.jsonl")

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 4763
    })
})


In [5]:
tokenizer.pad_token = tokenizer.eos_token

In [6]:
def tokenize_function(data):
    print(data)
    input = data['source']
    output = data['target']
    tokenized_inputs = tokenizer(input, truncation=True, padding='max_length', max_length=128)
    tokenized_outputs = tokenizer(output, truncation=True, padding='max_length', max_length=128)
    tokenized_inputs['labels'] = tokenized_outputs['input_ids']

    return tokenized_inputs



In [7]:
tokenized_dataset = dataset['train'].map(tokenize_function)

In [8]:
print(tokenized_dataset)

Dataset({
    features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4763
})


In [9]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

In [10]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3810
    })
    test: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 953
    })
})


In [11]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [12]:
from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
    output_dir="./finetuned-hindi",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2, 
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    num_train_epochs=2,
    fp16=True,
    logging_steps=50,
    save_strategy="epoch",
    eval_strategy="epoch",
    report_to="none",
)

In [13]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    # data_collator=data_collator
)

# Start fine-tuning
trainer.train()

# Save fine-tuned model
trainer.save_model("./finetuned-hindi-model")
tokenizer.save_pretrained("./finetuned-hindi-model")

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 